In [1]:
from hhop import DFExtender, SchemaManager, spark, col, F
import shutil, os, time # working with FS

from funs import read_table, write_table_through_view # useful functions
from functools import reduce

22/12/17 14:35:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/17 14:35:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Возможно, пригодятся 
1. .config("spark.hadoop.hive.exec.dynamic.partition", "true") \
2. .config("spark.hadoop.hive.exec.dynamic.partition.mode", "nonstrict") \

## Comparing tables

In [2]:
# df_ref_csv = spark.read.csv('./synth_data/table2.csv', header=True, sep=';')
# (
#     df_ref_csv.write.mode('overwrite')
#     .partitionBy('dt_part', 'group_part')
#     .saveAsTable('default.part_table_test2')
# )

df = read_table('default.nonpart_table_test1', alias='main')
df_ref = read_table('default.part_table_test2', alias='ref')

In [3]:
df

DataFrame[index: string, pk1: string, pk2: string, var1: string, var2: string, dt_part: string, group_part: string]

In [4]:
df_main = DFExtender(df)

In [5]:
df_main.compareTables(df_ref, key=['pk1', 'pk2'])

[[9, 8, 1], [9, 9, 0]]
{'var1_sum_error': 0, 'index_sum_error': 1, 'group_part_sum_error': 1, 'dt_part_sum_error': 2, 'var2_sum_error': 0}
{'not in main': 4, 'not in reference': 3, 'correct matching': 6}


In [180]:
df_ref

DataFrame[index: string, pk1: string, pk2: string, var1: string, var2: string, dt_part: string, group_part: string]

In [181]:
def add_column_is_diff(df, col):
    cond_diff = (
        (df1[dummy1].isNotNull() & df2[dummy2].isNotNull())
        & (df1[col] != df2[col])
    )
    return df.withColumn(col+diff_postfix, F.when(cond_diff, 1).otherwise(0))

In [211]:
df1_cols = set(df.columns)
df2_cols = set(df_ref.columns)
key = ['pk1', 'pk2']
common_cols = (df1_cols & df2_cols) - set(key)

diff_postfix = '_is_diff'
sum_postfix = '_sum_error'

cols_not_in_main, cols_not_in_ref = df2_cols - df1_cols, df1_cols - df2_cols


dummy_column = 'hhop_const_value_column'
dummy1, dummy2 = dummy_column + '1', dummy_column + '2'

df1 = df.withColumn(dummy1, F.lit(1))
df2 = df_ref.withColumn(dummy2, F.lit(1))

df_joined = df1.join(df2, on=key, how='full')

df_temp = reduce(add_column_is_diff, common_cols, df_joined)

diff_results = (
    df_temp
    .agg(
        *(
            F.sum(col_is_diff + diff_postfix).alias(col_is_diff + sum_postfix)
        for col_is_diff in common_cols
        )
    ).collect()[0]
)

diff_results_dict = {}

for column in common_cols:
    diff_column = column + diff_postfix
    sum_column = column + sum_postfix
    diff_results_dict[sum_column] = diff_results[sum_column]
    
print(diff_results_dict)

# 2 part
k = df_temp.groupBy(dummy1, dummy2).count().cache()

cases_full_join = {
    'not in main': (col(dummy1).isNull() & col(dummy2).isNotNull()),
    # col(dummy1).isNull() & col(dummy2).isNotNull()
    'not in reference': col(dummy1).isNotNull() & col(dummy2).isNull(),
    'correct matching': col(dummy1).isNotNull() & col(dummy2).isNotNull(),
}

get_cnt_cases = lambda x: k.filter(x).select('count').collect()[0]['count']

cnt_results = {}
for key, value in cases_full_join.items():
    cnt_results[key] = get_cnt_cases(value)
print(cnt_results)
# df_temp.show(100, 5)

{'var1_sum_error': 0, 'var2_sum_error': 0, 'dt_part_sum_error': 2, 'group_part_sum_error': 1, 'index_sum_error': 1}
{'not in main': 4, 'not in reference': 3, 'correct matching': 6}


In [187]:
df_temp.show(100, 5)

+----+----+-----+-----+-----+-------+----------+------------------------+-----+-----+-----+-------+----------+------------------------+------------+------------+---------------+------------------+-------------+
| pk1| pk2|index| var1| var2|dt_part|group_part|hhop_const_value_column1|index| var1| var2|dt_part|group_part|hhop_const_value_column2|var1_is_diff|var2_is_diff|dt_part_is_diff|group_part_is_diff|index_is_diff|
+----+----+-----+-----+-----+-------+----------+------------------------+-----+-----+-----+-------+----------+------------------------+------------+------------+---------------+------------------+-------------+
|key1|  11| null| null| null|   null|      null|                    null|    2|va...|va...|  20...|     gr...|                       1|           0|           0|              0|                 0|            0|
|key5|   5| null| null| null|   null|      null|                    null|    9|va...|va...|  20...|     gr...|                       1|           0|        

## Synth table

In [2]:
df_src = spark.read.csv('./synth_data/table1.csv', header=True, sep=';')

In [3]:
df_src.show()

+-----+----+----+------+--------+----------+----------+
|index| pk1| pk2|  var1|    var2|   dt_part|group_part|
+-----+----+----+------+--------+----------+----------+
|    1|key1|   1|value1|value2_1|2022-12-15|    group1|
|    2|key1|   1|value1|value2_1|2022-12-16|    group2|
|    3|key1|   2|value1|value2_1|2022-12-16|    group3|
|    4|key2|   2|  null|value2_1|2022-12-17|    group1|
|    5|key2|   3|value1|value2_1|2022-12-18|    group2|
|    6|key2|   4|value1|    null|2022-12-19|    group3|
|    7|key2|null|value1|value2_1|2022-12-19|    group4|
|    8|null|   4|value1|value2_1|2022-12-20|    group3|
|    9|null|null|value1|value2_1|2022-12-20|    group7|
+-----+----+----+------+--------+----------+----------+



In [4]:
df_src.write.mode('overwrite').partitionBy('dt_part', 'group_part').saveAsTable('default.part_table_test1')
df_src.write.mode('overwrite').saveAsTable('default.nonpart_table_test1')

## Info about table

### Reading table from Hive

In [5]:
df_hive = read_table('default.part_table_test1', verbose=True)

root
 |-- index: string (nullable = true)
 |-- pk1: string (nullable = true)
 |-- pk2: string (nullable = true)
 |-- var1: string (nullable = true)
 |-- var2: string (nullable = true)
 |-- dt_part: string (nullable = true)
 |-- group_part: string (nullable = true)

location: file:/Users/pyro/github/HiveHelper_on_PySpark/spark-warehouse/part_table_test1
partition columns: ['dt_part', 'group_part']


In [6]:
df = DFExtender(df_hive, pk=['pk1', 'pk2'], verbose=True)

### sanity checks: PK, NULL

In [7]:
df_filled = df_hive.fillna({'pk1': 'default', 'pk2': 'default'})

In [8]:
df_filled.show()

+-----+-------+-------+------+--------+----------+----------+
|index|    pk1|    pk2|  var1|    var2|   dt_part|group_part|
+-----+-------+-------+------+--------+----------+----------+
|    1|   key1|      1|value1|value2_1|2022-12-15|    group1|
|    2|   key1|      1|value1|value2_1|2022-12-16|    group2|
|    3|   key1|      2|value1|value2_1|2022-12-16|    group3|
|    5|   key2|      3|value1|value2_1|2022-12-18|    group2|
|    7|   key2|default|value1|value2_1|2022-12-19|    group4|
|    8|default|      4|value1|value2_1|2022-12-20|    group3|
|    4|   key2|      2|  null|value2_1|2022-12-17|    group1|
|    9|default|default|value1|value2_1|2022-12-20|    group7|
|    6|   key2|      4|value1|    null|2022-12-19|    group3|
+-----+-------+-------+------+--------+----------+----------+



In [9]:
DFExtender(df_filled, ['pk1', 'pk2'], verbose=True).getInfo()

Count all:           9
Unique PK count:     8
PK with duplicates:  1

Null values in columns - {'column': [count NULL, share NULL]}:
{'var1': [1, 0.1111], 'var2': [1, 0.1111]}


In [10]:
DFExtender(df_hive, ['pk1', 'pk2'], verbose=True).getInfo()

Count all:           9
PK column 'pk1' contains empty values, be careful!
PK column 'pk2' contains empty values, be careful!
Unique PK count:     8
PK with duplicates:  1

Null values in columns - {'column': [count NULL, share NULL]}:
{'pk1': [2, 0.2222], 'pk2': [2, 0.2222], 'var1': [1, 0.1111], 'var2': [1, 0.1111]}


In [11]:
df.groupBy(['pk1', 'pk2']).count().show()

+----+----+-----+
| pk1| pk2|count|
+----+----+-----+
|null|null|    1|
|key1|   1|    2|
|key2|   2|    1|
|key2|   3|    1|
|key2|null|    1|
|key1|   2|    1|
|null|   4|    1|
|key2|   4|    1|
+----+----+-----+



## SchemaManager

### dropping empty tables

In [12]:
schema_name = 'popular_schema'
spark.sql(f"drop database if exists {schema_name} cascade")
spark.sql(f'create database {schema_name}')

DataFrame[]

In [13]:
df_src_write = df_src.write.mode('overwrite')
df_src_write.partitionBy('dt_part', 'group_part').saveAsTable(f'{schema_name}.table1')
df_src_write.saveAsTable(f'{schema_name}.table2')
df_src_write.saveAsTable(f'{schema_name}.table3')

table1 has dir + data   
table2 has only root dir    
table3 doesn't have any dir and data

In [14]:
shutil.rmtree('./spark-warehouse/popular_schema.db/table2', ignore_errors=True)
os.makedirs('./spark-warehouse/popular_schema.db/table2')
shutil.rmtree('./spark-warehouse/popular_schema.db/table3', ignore_errors=True)

In [15]:
spark.sql(f"show tables in {schema_name}").show(10, False)

+--------------+---------+-----------+
|database      |tableName|isTemporary|
+--------------+---------+-----------+
|popular_schema|table1   |false      |
|popular_schema|table2   |false      |
|popular_schema|table3   |false      |
+--------------+---------+-----------+



In [16]:
popular_schema = SchemaManager('popular_schema')

3 tables in popular_schema
run drop_empty_tables() to drop empty tables in popular_schema


In [17]:
popular_schema.drop_empty_tables()

2 going to be dropped out of 3 (66.67%). Schema: popular_schema
After dropping tables there are 1 tables in popular_schema


22/12/16 16:57:54 ERROR FileUtils: Failed to delete file:/Users/pyro/github/HiveHelper_on_PySpark/spark-warehouse/popular_schema.db/table3


In [19]:
spark.sql(f"show tables in {schema_name}").show(10, False)

+--------------+---------+-----------+
|database      |tableName|isTemporary|
+--------------+---------+-----------+
|popular_schema|table1   |false      |
+--------------+---------+-----------+



## Extra

unionAll

splitting big dataset if applicable

writing tables

## Modification

1. read as you like, change class to DFExtender to get stats
2. use all methods from PySpark as usual

In [170]:
values = [
    ("x","x"),
    ("x","y"),
    ("x",None),
    (None,"x"),
    (None,None),
]
columns = ['val1', 'val2']
df_test = spark.createDataFrame(values, columns)

In [174]:
cond_diff = (
        (df_test['val1'] != df_test['val2'])
)

(
    df_test
    .withColumn(diff_postfix, F.when(cond_diff, 1).otherwise(0))
    .withColumn('cond', df_test['val1'].isNotNull())
    .show(10)
)

+----+----+--------+-----+
|val1|val2|_is_diff| cond|
+----+----+--------+-----+
|   x|   x|       0| true|
|   x|   y|       1| true|
|   x|null|       0| true|
|null|   x|       0|false|
|null|null|       0|false|
+----+----+--------+-----+

